In [5]:
from janome.tokenizer import Tokenizer
j_t = Tokenizer()
def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]

In [6]:
tokenizer_janome('コーラが飲みたいです。')

['コーラ', 'が', '飲み', 'たい', 'です', '。']

In [7]:
import re

In [8]:
def preprocessing_text(text):
    text = re.sub('/r', '', text)
    text = re.sub('/n', '', text)
    text = re.sub(' ', '', text)
    text = re.sub('　', '', text)
    
    text = re.sub(r'[0-9 ０-９]', '0', text)
    
    return text

In [11]:
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_janome(text)
    return ret

In [12]:
text = '昨日は とても暑く、気温が36度もあった。'
print(tokenizer_with_preprocessing(text))

['昨日', 'は', 'とても', '暑く', '、', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


In [13]:
import torchtext

In [14]:
max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, 
                            use_vocab=True, lower=True, include_lengths=True,
                           batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [18]:
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

print('訓練データの数', len(train_ds))
print('１つ目の訓練データ', vars(train_ds[0]))
print('2つ目の訓練データ', vars(train_ds[1]))

訓練データの数 4
１つ目の訓練データ {'Text': ['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。'], 'Label': '0'}
2つ目の訓練データ {'Text': ['機械', '学習', 'が', '好き', 'です', '。'], 'Label': '1'}


In [19]:
TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.freqs

Counter({'王': 1,
         'と': 5,
         '王子': 1,
         '女王': 1,
         '姫': 1,
         '男性': 1,
         '女性': 1,
         'が': 3,
         'い': 1,
         'まし': 1,
         'た': 1,
         '。': 4,
         '機械': 1,
         '学習': 1,
         '好き': 1,
         'です': 1,
         '本章': 2,
         'から': 1,
         '自然': 1,
         '言語': 1,
         '処理': 1,
         'に': 1,
         '取り組み': 1,
         'ます': 2,
         'で': 1,
         'は': 1,
         '商品': 1,
         'レビュー': 1,
         'の': 4,
         '短い': 1,
         '文章': 4,
         'に対して': 1,
         '、': 3,
         'その': 1,
         'ネガティブ': 1,
         'な': 4,
         '評価': 2,
         'を': 3,
         'し': 3,
         'て': 2,
         'いる': 2,
         'か': 2,
         'ポジティブ': 1,
         '0': 1,
         '値': 1,
         'クラス': 1,
         '分類': 2,
         'する': 1,
         'モデル': 1,
         '構築': 1})

In [20]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f1da6df0d90>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
     

In [30]:
train_dl = torchtext.data.Iterator(train_ds, batch_size=2, train=True)

val_dl = torchtext.data.Iterator(val_ds, batch_size=2, train=False, sort=False)

test_dl = torchtext.data.Iterator(test_ds, batch_size=2, train=False, sort=False)

In [31]:
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[46,  2, 47,  2, 40,  2, 42,  2, 48,  2, 39,  8, 19, 29, 23,  3,  1,  1,
          1,  1,  1,  1,  1,  1,  1],
        [45, 43,  8, 41, 25,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1]]), tensor([16,  6]))
tensor([0, 1])
